In [1]:
# Make necessary imports
from pyspark import SparkConf
from pyspark.context import SparkContext

In [2]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

22/02/11 04:00:44 WARN Utils: Your hostname, Kamiels-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 129.104.243.21 instead (on interface en0)
22/02/11 04:00:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/11 04:00:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
class MRAlgorithm:
    def __init__(self, datapath):
        self.sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
        self.input = self.sc.textFile(datapath).map(lambda x: (int(x.split(',')[0]),
                                                               int(x.split(',')[1]),int(x.split(',')[2])))
        
    def compute_W(self):
        print("Compute W")
        
    def compute_H(self):
        print("Compute H")
        
        # Define dummy H to continue with next operations
        self.H = self.sc.parallelize([(1, [0.2,0.3,0.5]), (2, [0.1,0.7,0.2]), (3, [0.4,0.3,0.3]), (4, [0.5,0.1,0.4])])
        
    def assign_clusters(self):
        #M10: Map user to cluster with highest probability
        self.clusters = self.H.map(lambda x: (x[0], x[1].index(max(x[1]))))
        
        #M11: Emit a 1 for each user that is in a cluster
        self.clustersizes = self.clusters.map(lambda x: (x[1], 1))
        
        #R11: Count the number of users per cluster
        self.clustersizes = self.clustersizes.reduceByKey(lambda x,y: x+y)
        
    def RM2_distribution(self):
        #M12: Emit each rating that a user gave
        self.ratings = self.input.map(lambda x: (x[1], x[2]))
        
        #R12: Sum ratings given by user
        self.ratings = self.ratings.reduceByKey(lambda x,y: x+y)

In [4]:
Algorithm = MRAlgorithm('../data/small_dataset.txt')
Algorithm.compute_H()
Algorithm.assign_clusters()
Algorithm.RM2_distribution()

Compute H


In [5]:
Algorithm.input.take(10)

[(1, 1488844, 3),
 (1, 822109, 5),
 (1, 885013, 4),
 (1, 30878, 4),
 (1, 823519, 3),
 (1, 893988, 3),
 (1, 124105, 4),
 (1, 1248029, 3),
 (1, 1842128, 4),
 (1, 2238063, 3)]

In [6]:
Algorithm.clusters.collect()

[(1, 2), (2, 1), (3, 0), (4, 0)]

In [7]:
Algorithm.clustersizes.collect()

[(0, 2), (1, 1), (2, 1)]

In [8]:
Algorithm.ratings.take(5)

[(1488844, 9), (30878, 8), (893988, 3), (1842128, 4), (2207774, 5)]